# Testing the branching algorithm

Following https://doi.org/10.1021/acs.jctc.7b00173 , compute the angle distributions.

Begin by testing a very flexible molecule as follows:

      1
      |
      0
     /  \
    2    3

Where the equilibrium angles are all 60 degrees but the harmonic angle potential is not strong. The bonds are rigid with a length of 2 and the diameters of the beads are 1.

In [4]:
import sys
import subprocess
import argparse
import random
import unittest
import numpy as np
import pandas as pd

params = {
    "prefix": "flexible_3_branch",
    "cubic_side_length": 90, "beta": 1,
    "trials_per": 1e5, "seed": random.randrange(int(1e9)),
    "production": 1e5}

with open(params['prefix']+'.fstprt', 'w') as file1:
    file1.write("""
4 sites
3 bonds
3 angles

1 site types
1 bond types
1 angle types

Site Properties

0 epsilon 1 sigma 1 cutoff 1

Sites

0 0 0 0 0
1 0 1 0 0
2 0 0 1 0
3 0 0 0 1

Bonds

0 0 0 1
0 0 0 2
0 0 0 3

Bond Properties

0 RigidBond length 1 delta 0.0001

Angles

0 0 1 0 2
0 0 1 0 3
0 0 2 0 3

Angle Properties

0 AngleHarmonic equilibrium_degrees 60 k_energy_per_radian_sq 1
""")

with open(params['prefix']+'_grow.txt', 'w') as file1:
    file1.write("""TrialGrowFile

particle_type 0 branch true mobile_site 2 mobile_site2 3 anchor_site 0 anchor_site2 1
""")

# write fst script to run a single simulation
with open('script.txt', "w") as myfile: myfile.write("""
MonteCarlo
RandomMT19937 seed {seed}
Configuration cubic_side_length 30 add_particles_of_type0 1 particle_type0 {prefix}.fstprt
Potential Model HardSphere
Potential Model HardSphere VisitModel VisitModelIntraMap exclude_bonds true
ThermoParams beta {beta} chemical_potential 1
Metropolis
TrialGrowFile grow_file {prefix}_grow.txt
Movie trials_per_write {trials_per} output_file {prefix}.xyz
Log trials_per_write {trials_per} output_file {prefix}.txt include_bonds true
Energy trials_per_write {trials_per} output_file {prefix}_en.txt
AnalyzeBonds trials_per_write {trials_per} output_file {prefix}_bonds.txt
CheckEnergy trials_per_update {trials_per} tolerance 1e-8
Run num_trials {production}
""".format(**params))

import subprocess
syscode = subprocess.call("../../../build/bin/fst < script.txt > script.log", shell=True, executable='/bin/bash')
with open('script.log', 'r') as file: print(file.read(), '\n', 'exit:', syscode)

# FEASST version: v0.24.5-3-g7548e16503-hwh/srswmonoclinic4
MonteCarlo
RandomMT19937 seed 479563718  
# initializing random number generator with seed: 479563718
Configuration add_particles_of_type0 1 cubic_side_length 30 particle_type0 flexible_3_branch.fstprt  
Potential Model HardSphere  
Potential Model HardSphere VisitModel VisitModelIntraMap exclude_bonds true  
ThermoParams beta 1 chemical_potential 1  
Metropolis  
TrialGrowFile grow_file flexible_3_branch_grow.txt  
Movie output_file flexible_3_branch.xyz trials_per_write 100000.0  
Log include_bonds true output_file flexible_3_branch.txt trials_per_write 100000.0  
Energy output_file flexible_3_branch_en.txt trials_per_write 100000.0  
AnalyzeBonds output_file flexible_3_branch_bonds.txt trials_per_write 100000.0  
CheckEnergy tolerance 1e-8 trials_per_update 100000.0  
Run num_trials 100000.0  
 
 exit: 0


In [2]:
def rg_fit(beta, beta1, rg1, der1):
    return rg1 + (beta - beta1)*der1

class TestFreelyJointedIdealChain(unittest.TestCase):
    def test(self):
        taylor2001 = pd.read_csv('../test/data/taylor2001rg.csv', header=None)
        #print(taylor2001)
        rgdf = pd.read_csv('chain5rg.txt')
        endf = pd.read_csv('chain5en.txt')
        rg = rgdf['average'].values[0]
        en = endf['average'].values[0]
        self.assertAlmostEqual(rg, np.sqrt(0.88), delta=0.01)
        self.assertAlmostEqual(en, -2.04, delta=0.06)
        rg_en = rgdf['rgu'].values[0]
        drg_dbeta = rg*en - rg_en
        self.assertAlmostEqual(drg_dbeta, -0.186/2, delta=0.01)
        print('drg_dbeta', drg_dbeta)

#         import matplotlib.pyplot as plt
#         plt.scatter(params['beta'], rg, label='explicit sim')
#         plt.plot(1./taylor2001[0], np.sqrt(taylor2001[1]), label='taylor2001')
#         plt.plot(1./taylor2001[0],
#                  rg_fit(beta=1./taylor2001[0],
#                         beta1=params['beta'],
#                         rg1=rg,
#                         der1=drg_dbeta),
#                  label='1st order extrapolation')
#         plt.xscale('log')
#         plt.ylim([0.2, 1.3])
#         plt.legend()
#         plt.xlabel(r'$\beta$', fontsize=16)
#         plt.ylabel(r'$R_g$', fontsize=16)
#         plt.show()

        
unittest.main(argv=[''], verbosity=2, exit=False)

test (__main__.TestFreelyJointedIdealChain) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.003s

OK


drg_dbeta -0.09925819217655563


Did this tutorial work as expected? Did you find any inconsistencies or have any comments? Please [contact](../../../CONTACT.rst) us. Any feedback is appreciated!